### Install packages

Run `pip install -r requirements.txt` to install the required packages.

train yolov with the custom dataset

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolo11l-cls.pt')  # load a pretrained model (recommended for training)

# Train the model
results = model.train(
    data="dataset", epochs=200,
    # adjust batch size based on GPU memory
    batch=150,
    patience=10,
    hsv_h=0.02, hsv_s=0.4, hsv_v=0.4,
    # disable random augmentations
    fliplr=0, scale=0, erasing=0,
    translate=0, degrees=0,
    mixup=0.1, cutmix=0.1,
)

validate data using trained weight

In [ ]:
from ultralytics import YOLO

# Load a pretrained model
model = YOLO(str(results.save_dir)+'/weights/best.pt')

source = 'dataset/test/**/*.jpg'

# Run inference on the source
validateResults = model(source, stream=True,)  # generator of Results objects

for result in validateResults:
    print(result.probs.top1)

export ONNX nodel

In [ ]:
from ultralytics import YOLO

model = YOLO(str(results.save_dir)+'/weights/best.pt')
model.export(format='onnx')